In [ ]:
import pandas as pd
import numpy as np 
import plotly.express as px 
from urllib.request import urlopen
import json

In [ ]:
covid19_data = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-20-2020.csv"
df_covid19_confirmed = pd.read_csv(covid19_data)
df_covid19_confirmed = df_covid19_confirmed[df_covid19_confirmed["Country_Region"]== "US"]
df_covid19_confirmed = df_covid19_confirmed.iloc[:, [0,7,10]]
df_covid19_confirmed = df_covid19_confirmed[df_covid19_confirmed["FIPS"].notnull()]
df_covid19_confirmed["FIPS"] = df_covid19_confirmed["FIPS"].astype(int)
df_covid19_confirmed["FIPS"] = df_covid19_confirmed["FIPS"].astype(str)
df_covid19_confirmed["FIPS"] = df_covid19_confirmed["FIPS"].str.zfill(5)
df_covid19_confirmed.sort_values('FIPS')
df_covid19_confirmed.head()

In [ ]:
df_hospitalBed_data = pd.read_csv("Hospitals.csv")
#df_hospitalBed_data.sort_values(by=['CITY'], ascending=True)
df_hospitalBed_data = df_hospitalBed_data.iloc[:, [15, 31]]
df_hospitalBed_data.dropna(inplace=True)
df_hospitalBed_data = df_hospitalBed_data[df_hospitalBed_data != "NOT AVAILABLE"]
df_hospitalBed_data = df_hospitalBed_data[df_hospitalBed_data["COUNTYFIPS"].notnull()]
df_hospitalBed_data["COUNTYFIPS"] = pd.to_numeric(df_hospitalBed_data["COUNTYFIPS"], downcast="float")
df_hospitalBed_data["COUNTYFIPS"] = df_hospitalBed_data["COUNTYFIPS"].astype(int)
df_hospitalBed_data["COUNTYFIPS"] = df_hospitalBed_data["COUNTYFIPS"].astype(str)
df_hospitalBed_data["COUNTYFIPS"] = df_hospitalBed_data["COUNTYFIPS"].str.zfill(5)
df_hospitalBed_data = df_hospitalBed_data.rename(columns={"COUNTYFIPS": "FIPS"})
df_hospitalBed_data = df_hospitalBed_data.set_index('FIPS').sum(level=0).reset_index()
df_hospitalBed_data.head()

In [ ]:
df_resource_data = pd.merge(df_covid19_confirmed,
                           df_hospitalBed_data[["FIPS","BEDS"]],
                           on = "FIPS",
                           how = "left")
df_resource_data = df_resource_data[df_resource_data["BEDS"].notnull()]
df_resource_data.head()

In [ ]:
df_resource_data["Total Number Bed Needed"] = -1*((df_resource_data["BEDS"]-df_resource_data["Confirmed"]))
df_resource_data["Active Number Bed Needed"] = -1*((df_resource_data["BEDS"]-df_resource_data["Active"]))
df_resource_data.head()

In [ ]:
import plotly.figure_factory as ff
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]
endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = df_resource_data['FIPS'].tolist()
values = df_resource_data['Total Number Bed Needed'].tolist()


fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    asp = 2.9,
    title_text = 'Bed Shortage Across US',
    legend_title = 'Total Number Bed Needed'
)
fig.layout.template = None
fig.show()

In [ ]:
df_resource_data.to_csv('HW1.zip')  